### 1、tensorflow 1.x 保存 MNIST 分类模型

本文档在 tensorflow 1.x 的环境下训练一个mnist数据集分类模型，并保存训练之后的模型，作为后续部署的模型，说明如下：  

**（1）**由于本项目主要是为了说明部署方法，所以这里采用简单的模型结果，复杂的模型在保存和部署上并没有什么区别  
**（2）**模型保存成 .pb　的格式，.pb模型通用性比较好，可以实现不同平台（serving、java、andriod等），具体参考[TensorFlow 保存模型为 PB 文件](https://zhuanlan.zhihu.com/p/32887066)  

tensorflow 中保存为 .pb 格式模型的方法有两种：  
**（1）** 保存为单个文件  
**（2）** 保存为文件夹  

下面分别说明保存方法和调用方法。

#### 1.1 第一种保存方法——保存为单个 .pb 文件
##### **1.1.1 模型训练和保存**

In [1]:
import tensorflow as tf
from tensorflow import graph_util # 保存模型用到的方法
from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test

# 模型参数
input_dim=784
hidden_dim=200
output_dim=10
learning_rate=0.5
train_steps=10000
batch_size=100

# 模型的输入，在调用部署后的模型时需要用到，所以这里最好自己命名
x = tf.placeholder(tf.float32, [None, input_dim], name='x-input')
y_ = tf.placeholder(tf.float32, [None, output_dim], name='y-input')

# 构建一个三层的全连接层网络，输入层、隐层和输出层分别为784,200,10
hidden = tf.layers.dense(inputs=x, units=hidden_dim, activation=tf.nn.relu)
y=tf.layers.dense(inputs=hidden,units=output_dim)

# 模型的预测输出，注意，因为我们部署之后希望拿到这个值，所以这里最好自己命名，后面部署之后的调用会用到
y_pred=tf.math.argmax(y,1,name="predict") 

# 模型损失
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy)

# 预测精度
correct_predict=tf.equal(tf.math.argmax(y,1),tf.math.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#模型训练
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validation_feed={x:mnist_validation.images,y_:mnist_validation.labels}
    test_feed={x:mnist_test.images,y_:mnist_test.labels}
    for i in range(train_steps):
        train_samples,train_labels=mnist_train.next_batch(batch_size)
        _,train_loss=sess.run([train_step,loss],feed_dict={x:train_samples,y_:train_labels})
        if i%1000==0:
            val_acc=sess.run(accuracy,feed_dict=validation_feed)
            print("after %d training steps, loss of train set is %f,validation accuracy is %f"%(i,train_loss,val_acc))
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("test accuracy is %f"%test_acc)
    
    # 保存模型
    # 将权重和偏置等变量转换为常量，然后将计算图保存，可以大大减小保存后的模型
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def,output_node_names=["predict"]) 
    model_save_path="model_save/method_1/mnist_classification.pb"
    with tf.gfile.FastGFile(model_save_path, mode='wb') as f:
        f.write(constant_graph.SerializeToString())

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
after 0 training steps, loss of train set is 2.414426,validation accuracy is 0.355400
after 1000 training steps, loss of train set is 0.192877,validation accuracy is 0.97

##### **1.1.2 模型的调用** 

In [ ]:
# 在python 中调用保存好的模型进行预测
import tensorflow as tf
from tensorflow.python.platform import gfile

from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test


model_save_path="model_save/method_1/mnist_classification.pb"

with tf.Session() as sess:
    # 读取保存的模型
    with gfile.FastGFile(model_save_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def,name='') # 这里的 name 不设置的话，系统默认为 import,则后面 get_tensor_by_name 需要在名称之前加import，如 "import/predict:0" 
    for i,n in enumerate(graph_def.node): # 假如不知道各层的名称，通过打印获取
        print("\tName of the node - %s"%n.name)
    g = tf.get_default_graph()
    y_pred=g.get_tensor_by_name("predict:0")
    x = g.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

# 假如报 Attempting to use uninitialized value 的错误，是因为训练的session 影响了这里的session，
# 重启一下 kernel,然后再跑这个调用部分即可

#### 1.2 第2种保存方法——保存为文件夹
##### **1.2.1 模型训练和保存**

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test

# 模型参数
input_dim=784
hidden_dim=200
output_dim=10
learning_rate=0.5
train_steps=10000
batch_size=100

# 模型的输入，在调用部署后的模型时需要用到，所以这里最好自己命名
x = tf.placeholder(tf.float32, [None, input_dim], name='x-input')
y_ = tf.placeholder(tf.float32, [None, output_dim], name='y-input')

# 构建一个三层的全连接层网络，输入层、隐层和输出层分别为784,200,10
hidden = tf.layers.dense(inputs=x, units=hidden_dim, activation=tf.nn.relu)
y=tf.layers.dense(inputs=hidden,units=output_dim)

# 模型的预测输出，注意，因为我们部署之后希望拿到这个值，所以这里最好自己命名，后面部署之后的调用会用到
y_pred=tf.math.argmax(y,1,name="predict") 

# 模型损失
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy)

# 预测精度
correct_predict=tf.equal(tf.math.argmax(y,1),tf.math.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#模型训练
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validation_feed={x:mnist_validation.images,y_:mnist_validation.labels}
    test_feed={x:mnist_test.images,y_:mnist_test.labels}
    for i in range(train_steps):
        train_samples,train_labels=mnist_train.next_batch(batch_size)
        _,train_loss=sess.run([train_step,loss],feed_dict={x:train_samples,y_:train_labels})
        if i%1000==0:
            val_acc=sess.run(accuracy,feed_dict=validation_feed)
            print("after %d training steps, loss of train set is %f,validation accuracy is %f"%(i,train_loss,val_acc))
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("test accuracy is %f"%test_acc)
    
    # 保存模型
    builder = tf.saved_model.builder.SavedModelBuilder('model_save/method_2/')
    # SavedModelBuilder里面放的是你想要保存的路径，比如我的路径是根目录下的dense_model文件
    builder.add_meta_graph_and_variables(sess, ["serve"])
    # 第二步必需要有，它是给你的模型贴上一个标签，这样再次调用的时候就可以根据标签来找。我给它起的标签名是"serve"，你也可以起别的名字，不过你需要记住你起的名字是什么。
    builder.save()

##### **1.2.2 模型的调用**

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], 'model_save/method_2')
    y_pred=sess.graph.get_tensor_by_name("predict:0")
    x = sess.graph.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], 'model_save/method_2')
    y_pred=sess.graph.get_tensor_by_name("predict:0")
    x = sess.graph.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

#### 1.2 第2种保存方法——保存为文件夹
##### **1.2.1 模型训练和保存**

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test

# 模型参数
input_dim=784
hidden_dim=200
output_dim=10
learning_rate=0.5
train_steps=10000
batch_size=100

# 模型的输入，在调用部署后的模型时需要用到，所以这里最好自己命名
x = tf.placeholder(tf.float32, [None, input_dim], name='x-input')
y_ = tf.placeholder(tf.float32, [None, output_dim], name='y-input')

# 构建一个三层的全连接层网络，输入层、隐层和输出层分别为784,200,10
hidden = tf.layers.dense(inputs=x, units=hidden_dim, activation=tf.nn.relu)
y=tf.layers.dense(inputs=hidden,units=output_dim)

# 模型的预测输出，注意，因为我们部署之后希望拿到这个值，所以这里最好自己命名，后面部署之后的调用会用到
y_pred=tf.math.argmax(y,1,name="predict") 

# 模型损失
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy)

# 预测精度
correct_predict=tf.equal(tf.math.argmax(y,1),tf.math.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#模型训练
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validation_feed={x:mnist_validation.images,y_:mnist_validation.labels}
    test_feed={x:mnist_test.images,y_:mnist_test.labels}
    for i in range(train_steps):
        train_samples,train_labels=mnist_train.next_batch(batch_size)
        _,train_loss=sess.run([train_step,loss],feed_dict={x:train_samples,y_:train_labels})
        if i%1000==0:
            val_acc=sess.run(accuracy,feed_dict=validation_feed)
            print("after %d training steps, loss of train set is %f,validation accuracy is %f"%(i,train_loss,val_acc))
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("test accuracy is %f"%test_acc)
    
    # 保存模型
    builder = tf.saved_model.builder.SavedModelBuilder('model_save/method_2/')
    # SavedModelBuilder里面放的是你想要保存的路径，比如我的路径是根目录下的dense_model文件
    builder.add_meta_graph_and_variables(sess, ["serve"])
    # 第二步必需要有，它是给你的模型贴上一个标签，这样再次调用的时候就可以根据标签来找。我给它起的标签名是"serve"，你也可以起别的名字，不过你需要记住你起的名字是什么。
    builder.save()

##### **1.2.2 模型的调用**

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], 'model_save/method_2')
    y_pred=sess.graph.get_tensor_by_name("predict:0")
    x = sess.graph.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

In [11]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], 'model_save/method_2')
    y_pred=sess.graph.get_tensor_by_name("predict:0")
    x = sess.graph.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
	Name of the node - x-input
	Name of the node - dense/kernel
	Name of the node - dense/kernel/read
	Name of the node - dense/bias
	Name of the node - dense/bias/read
	Name of the node - dense/MatMul
	Name of the node - dense/BiasAdd
	Name of the node - dense/Relu
	Name of the node - dense_1/kernel
	Name of the node - dense_1/kernel/read
	Name of the node - dense_1/bias
	Name of the node - dense_1/bias/read
	Name of the node - dense_1/MatMul
	Name of the node - dense_1/BiasAdd
	Name of the node - predict/dimension
	Name of the node - predict
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[7 2 1 ... 4 5 6]


#### 1.2 第2种保存方法——保存为文件夹
##### **1.2.1 模型训练和保存**

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 获取 mnist数据
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
mnist_train,mnist_validation,mnist_test=mnist.train,mnist.validation,mnist.test

# 模型参数
input_dim=784
hidden_dim=200
output_dim=10
learning_rate=0.5
train_steps=10000
batch_size=100

# 模型的输入，在调用部署后的模型时需要用到，所以这里最好自己命名
x = tf.placeholder(tf.float32, [None, input_dim], name='x-input')
y_ = tf.placeholder(tf.float32, [None, output_dim], name='y-input')

# 构建一个三层的全连接层网络，输入层、隐层和输出层分别为784,200,10
hidden = tf.layers.dense(inputs=x, units=hidden_dim, activation=tf.nn.relu)
y=tf.layers.dense(inputs=hidden,units=output_dim)

# 模型的预测输出，注意，因为我们部署之后希望拿到这个值，所以这里最好自己命名，后面部署之后的调用会用到
y_pred=tf.math.argmax(y,1,name="predict") 

# 模型损失
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
loss = tf.reduce_mean(cross_entropy)

# 预测精度
correct_predict=tf.equal(tf.math.argmax(y,1),tf.math.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32))

#模型训练
train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validation_feed={x:mnist_validation.images,y_:mnist_validation.labels}
    test_feed={x:mnist_test.images,y_:mnist_test.labels}
    for i in range(train_steps):
        train_samples,train_labels=mnist_train.next_batch(batch_size)
        _,train_loss=sess.run([train_step,loss],feed_dict={x:train_samples,y_:train_labels})
        if i%1000==0:
            val_acc=sess.run(accuracy,feed_dict=validation_feed)
            print("after %d training steps, loss of train set is %f,validation accuracy is %f"%(i,train_loss,val_acc))
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("test accuracy is %f"%test_acc)
    
    # 保存模型
    builder = tf.saved_model.builder.SavedModelBuilder('model_save/method_2/')
    # SavedModelBuilder里面放的是你想要保存的路径，比如我的路径是根目录下的dense_model文件
    builder.add_meta_graph_and_variables(sess, ["serve"])
    # 第二步必需要有，它是给你的模型贴上一个标签，这样再次调用的时候就可以根据标签来找。我给它起的标签名是"serve"，你也可以起别的名字，不过你需要记住你起的名字是什么。
    builder.save()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
after 0 training steps, loss of train set is 2.387276,validation accuracy is 0.381400
after 1000 training steps, loss of train set is 0.130141,validation accuracy is 0.969600
after 2000 training steps, loss of train set is 0.059555,validation accuracy is 0.973400
after 3000 training steps, loss of train set is 0.003633,validation accuracy is 0.980400
after 4000 training steps, loss of train set is 0.017264,validation accuracy is 0.981200
after 5000 training steps, loss of train set is 0.018133,validation accuracy is 0.980400
after 6000 training steps, loss of train set is 0.008507,validation accuracy is 0.983200
after 7000 training steps, loss of train set is 0.006637,validation accuracy is 0.982600
after 8000 training steps, loss of train set is 0.005118,validation accuracy is 0.983200
after 90

##### **1.2.2 模型的调用**

In [8]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ['serve'], 'model_save/method_2')
    y_pred=sess.graph.get_tensor_by_name("predict:0")
    x = sess.graph.get_tensor_by_name("x-input:0")
    y_= sess.run(y_pred, feed_dict={x: mnist_test.images})
    print(mnist_test.labels)
    print(y_)

INFO:tensorflow:Restoring parameters from model_save/method_2/variables/variables
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[7 2 1 ... 4 5 6]
